# Imports

In [1]:
import sys

sys.path.append("..")
from brazilian_data import EconomicDataAsync
import nest_asyncio
nest_asyncio.apply()

# Definition of the codes for search

**The variable start_date defines the initial date for data collection.**

In [2]:
DATA_INICIO = "2000-01-01"

**The Banco Central codes should be a dictionary with the column name that the `datas_banco_central` function will recognize as the column name and the series code.**

```

variaveis_banco_central= {
    "selic": 4189,
    "cambio": 3698,
    "pib_mensal": 4380,
    "igp_m": 189,
    "igp_di": 190,
    "m1": 27788,
}
```

In [3]:
variaveis_banco_central= {
    "selic": 4189,
    "cambio": 3698,
    "pib_mensal": 4380,
    "igp_m": 189,
    "igp_di": 190,
    "m1": 27788,
}

The `ibge_codes` input should be a dictionary where each key represents the name of a variable (e.g., "ipca"), which the function will use as the column name. The value associated with each key should be another dictionary containing the following fields:

- `"codigo"`: An integer representing the code of the data series to be collected. This code is specific to the variable and identifies the series in the IBGE database.

- `"territorial_level"`: A string indicating the territorial level of the data, such as "1" for Brazil, "2" for regions, "3" for states, etc.

- `"ibge_territorial_code"`: A code defining the specific geographical area of the data. The value `"all"` indicates that data for all areas corresponding to the territorial level should be collected.

- `"variable"`: A string or number that identifies the specific variable within the data series. This may represent a specific category or indicator to be collected.

Example:

```
variaveis_ibge = {
    "ipca": {
        "codigo": 1737,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "63",
    }
}
```

In [4]:
variaveis_ibge = {
    "ipca": {
        "codigo": 1737,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "63",
    },
    "custo_m2": {
        "codigo": 2296,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",
    },
    "pesquisa_industrial_mensal": {
        "codigo": 8159,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",
    },
    "pmc_volume": {
        "codigo": 8186,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11709",
    },
}

The `codes ibge link` input should be a dictionary where each key represents the name of an economic indicator or specific variable (e.g., "pib", "soja"), which the function will use as the column name. The value associated with each key is a URL that points to an Excel file available on the IBGE website, containing the data corresponding to that indicator.

These URLs are dynamically generated from the IBGE SIDRA system and can be used to download the tables directly. Each link contains specific parameters defining the selection of variables, periods, and territorial levels relevant to the query.

Example:

```
    {"despesas_publica": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93405/d/v6561%201/l/v,p%2Bc11255,t",
    "capital_fixo": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93406/d/v6561%201/l/v,p%2Bc11255,t",
    "producao_industrial_manufatureira": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8158.xlsx&terr=N&rank=-&query=t/8158/n1/all/v/11599/p/all/c543/129278/d/v11599%205/l/v,p%2Bc543,t",
    "soja": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39443/l/v,p%2Bc48,t",}
```

These URLs allow for the direct download of the data in Excel format, which can then be processed by your code.

In [5]:
indicadores_ibge_link = {
    "pib": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6564/p/all/c11255/90707/d/v6564%201/l/v,p,t%2Bc11255&verUFs=false&verComplementos2=false&verComplementos1=false&omitirIndentacao=false&abreviarRotulos=false&exibirNotas=false&agruparNoCabecalho=false",
    "despesas_publica": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93405/d/v6561%201/l/v,p%2Bc11255,t",
    "capital_fixo": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93406/d/v6561%201/l/v,p%2Bc11255,t",
    "producao_industrial_manufatureira": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8158.xlsx&terr=N&rank=-&query=t/8158/n1/all/v/11599/p/all/c543/129278/d/v11599%205/l/v,p%2Bc543,t",
    "soja": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39443/l/v,p%2Bc48,t",
    "milho_1": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39441/l/v,p%2Bc48,t",
    "milho_2": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39442/l/v,p%2Bc48,t",
}

**The `IPEADATA codes` should be a dictionary with the column name that the `datas_IPEADATA` function will recognize as the column name and the series code.**

```

codigos_ipeadata= {
    "taja_juros_ltn": "ANBIMA12_TJTLN1212",
    "imposto_renda": "SRF12_IR12",
    "ibovespa": "ANBIMA12_IBVSP12",
    "consumo_energia": "ELETRO12_CEET12",
    "brent_fob": "EIA366_PBRENT366",
}
```

In [6]:
codigos_ipeadata= {
    "taja_juros_ltn": "ANBIMA12_TJTLN1212",
    "imposto_renda": "SRF12_IR12",
    "ibovespa": "ANBIMA12_IBVSP12",
    "consumo_energia": "ELETRO12_CEET12",
    "brent_fob": "EIA366_PBRENT366",
}

The `codes fred` should be a dictionary with the column name that the `datas_fred` function will recognize as the column name and the series code.

```
codigos_fred = {
    "nasdaq100": "NASDAQ100",
    "taxa_cambio_efetiva": "RBBRBIS",
    "cboe_nasdaq": "VXNCLS",
    "taxa_juros_interbancaria": "IRSTCI01BRM156N",
    "atividade_economica_eua": "USPHCI",}

```

In [7]:

codigos_fred = {
    "nasdaq100": "NASDAQ100",
    "taxa_cambio_efetiva": "RBBRBIS",
    "cboe_nasdaq": "VXNCLS",
    "taxa_juros_interbancaria": "IRSTCI01BRM156N",
    "atividade_economica_eua": "USPHCI",
    "indice_confianca_manufatura": "BSCICP03BRM665S",
    "indice_confianca_exportadores": "BSXRLV02BRM086S",
    "indice_tendencia_emprego": "BRABREMFT02STSAM",
    "indice_confianca_consumidor": "CSCICP03BRM665S",
    "capacidade_instalada": "BSCURT02BRM160S",
}

# Metodo help

In [8]:
economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                 codes_ibge=variaveis_ibge, 
                                 codes_ipeadata=codigos_ipeadata, 
                                 codes_ibge_link=indicadores_ibge_link,
                                 start_date=DATA_INICIO)

In [9]:
economic_brazil.help()


        EconomicData Class Help:

        Methods:

        - data_index():
            Generate a DataFrame with a date range starting from start_date.

        - datas_banco_central_async(save=None, diretory=None, data_format=None):
            Fetch data from Banco Central and handle exceptions.
            
            Link for verification Time Series: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries  
            
            Input codes: 
                - The Banco Central codes should be a dictionary with the column name that the `datas_banco_central` function will recognize as the column name and the series code. Example: {"selic": 4189}
                
            
        - datas_ibge_async(save=False, diretory=None, data_format=None):
        
            Fetch IBGE data and handle exceptions.
            
            Link for verification Time Series: https://sidra.ibge.gov.br/home/ipca/brasil
            
            

# Method datas_brazil

## Without missing data

In [10]:
%%timeit
data_bcb = True
data_ibge = True
data_ibge_link = True
data_ipeadata = True
data_fred = False
economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                 codes_ibge=variaveis_ibge, 
                                 codes_ipeadata=codigos_ipeadata, 
                                 codes_ibge_link=indicadores_ibge_link,
                                 start_date=DATA_INICIO)

dados = economic_brazil.datas_brazil_async(datas_bcb= data_bcb,
                                     datas_ibge_codigos=data_ibge, 
                                     datas_ibge_link=data_ibge_link, 
                                     datas_ipeadata=data_ipeadata,
                                     missing_data=True)

Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.
Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'te

In [11]:
data_bcb = True
data_ibge = True
data_ibge_link = True
data_ipeadata = True
data_fred = False
economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                 codes_ibge=variaveis_ibge, 
                                 codes_ipeadata=codigos_ipeadata, 
                                 codes_ibge_link=indicadores_ibge_link,
                                 start_date=DATA_INICIO)

dados = economic_brazil.datas_brazil_async(datas_bcb= data_bcb,
                                     datas_ibge_codigos=data_ibge, 
                                     datas_ibge_link=data_ibge_link, 
                                     datas_ipeadata=data_ipeadata,
                                     missing_data=True)

Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.


In [12]:
dados.head()

,selic,cambio,pib_mensal,igp_m,igp_di,m1,ipca,custo_m2,pesquisa_industrial_mensal,pib,...,producao_industrial_manufatureira,soja,milho_1,milho_2,ipca_expectativa_focus,taja_juros_ltn,imposto_renda,ibovespa,consumo_energia,brent_fob
2000-01-01,18.94,1.8037,92576.6,1.24,1.02,79015372.0,0.62,5.51,69.71441,1.4,...,57.43586,52381672.0,31477232.0,10595844.0,0.522857,19.465261,5043.680936,-4.113276,25060.0,25.511000
2000-02-01,18.87,1.7753,91770.4,0.35,0.19,79015372.0,0.13,5.51,69.71441,1.4,...,57.43586,52381672.0,31477232.0,10595844.0,0.531897,19.465261,4120.602582,7.761777,25057.0,27.775714
2000-03-01,18.85,1.7420,92579.9,0.15,0.18,79015372.0,0.22,5.51,69.71441,1.1,...,57.43586,52381672.0,31477232.0,10595844.0,0.593452,19.465261,5606.185192,0.906002,25662.0,27.486087
2000-04-01,18.62,1.7682,91376.2,0.23,0.13,79015372.0,0.42,5.51,69.71441,1.1,...,57.43586,52381672.0,31477232.0,10595844.0,0.587363,19.465261,4634.431697,-12.811448,25598.0,22.764444
2000-05-01,18.51,1.8279,98727.0,0.31,0.67,79015372.0,0.01,5.51,69.71441,1.1,...,57.43586,52381672.0,31477232.0,10595844.0,0.562051,21.681500,4047.302075,-3.739461,25448.0,27.737619


## Save data

In [13]:

data_bcb = True
data_ibge = True
data_ibge_link = True
data_ipeadata = True
data_fred = False
economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                 codes_ibge=variaveis_ibge, 
                                 codes_ipeadata=codigos_ipeadata, 
                                 codes_ibge_link=indicadores_ibge_link,
                                 start_date=DATA_INICIO)

dados = economic_brazil.datas_brazil_async(datas_bcb= data_bcb,
                                     datas_ibge_codigos=data_ibge, 
                                     datas_ibge_link=data_ibge_link, 
                                     datas_ipeadata=data_ipeadata,
                                     save=True,
                                     directory="../dados/economicos_brazil",
                                     data_format="csv")

Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.


## With missing data

In [14]:
data_bcb = True
data_ibge = True
data_ibge_link = True
data_ipeadata = True
data_fred = False
economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                 codes_ibge=variaveis_ibge, 
                                 codes_ipeadata=codigos_ipeadata, 
                                 codes_ibge_link=indicadores_ibge_link,
                                 start_date=DATA_INICIO)

dados = economic_brazil.datas_brazil_async(datas_bcb= data_bcb,
                                     datas_ibge_codigos=data_ibge, 
                                     datas_ibge_link=data_ibge_link, 
                                     datas_ipeadata=data_ipeadata,
                                     missing_data=False)
dados.head()

Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.


,selic,cambio,pib_mensal,igp_m,igp_di,m1,ipca,custo_m2,pesquisa_industrial_mensal,pib,...,producao_industrial_manufatureira,soja,milho_1,milho_2,ipca_expectativa_focus,taja_juros_ltn,imposto_renda,ibovespa,consumo_energia,brent_fob
2000-01-01,18.94,1.8037,92576.6,1.24,1.02,NaN,0.62,NaN,NaN,1.4,...,NaN,NaN,NaN,NaN,0.522857,NaN,5043.680936,-4.113276,25060.0,25.511000
2000-02-01,18.87,1.7753,91770.4,0.35,0.19,NaN,0.13,NaN,NaN,1.4,...,NaN,NaN,NaN,NaN,0.531897,NaN,4120.602582,7.761777,25057.0,27.775714
2000-03-01,18.85,1.7420,92579.9,0.15,0.18,NaN,0.22,NaN,NaN,1.1,...,NaN,NaN,NaN,NaN,0.593452,NaN,5606.185192,0.906002,25662.0,27.486087
2000-04-01,18.62,1.7682,91376.2,0.23,0.13,NaN,0.42,NaN,NaN,1.1,...,NaN,NaN,NaN,NaN,0.587363,19.465261,4634.431697,-12.811448,25598.0,22.764444
2000-05-01,18.51,1.8279,98727.0,0.31,0.67,NaN,0.01,NaN,NaN,1.1,...,NaN,NaN,NaN,NaN,0.562051,21.681500,4047.302075,-3.739461,25448.0,27.737619


# Method datas_banco_central

In [15]:
DATA_INICIO = "2000-01-01"
variaveis_banco_central= {
    "selic": 4189,
    "cambio": 3698,
    "pib_mensal": 4380,
    "igp_m": 189,
    "igp_di": 190,
    "m1": 27788,
}

economic_brazil = EconomicDataAsync(codes_banco_central=variaveis_banco_central, 
                                start_date=DATA_INICIO)

dados = economic_brazil.datas_banco_central_async()

In [16]:
dados.head()

,selic,cambio,pib_mensal,igp_m,igp_di,m1
Date,,,,,,
2000-01-01,18.94,1.8037,92576.6,1.24,1.02,NaN
2000-02-01,18.87,1.7753,91770.4,0.35,0.19,NaN
2000-03-01,18.85,1.7420,92579.9,0.15,0.18,NaN
2000-04-01,18.62,1.7682,91376.2,0.23,0.13,NaN
2000-05-01,18.51,1.8279,98727.0,0.31,0.67,NaN


# Method datas_ibge

In [17]:
variaveis_ibge = {
    "ipca": {
        "codigo": 1737,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "63",
    },
    "custo_m2": {
        "codigo": 2296,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",
    },
    "pesquisa_industrial_mensal": {
        "codigo": 8159,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",
    },
    "pmc_volume": {
        "codigo": 8186,
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11709",
    },
}

In [18]:
economic_brazil = EconomicDataAsync(codes_ibge=variaveis_ibge, 
                                start_date=DATA_INICIO)

dados = economic_brazil.datas_ibge_async()

dados.head()

Erro na coleta de dados da variável pmc_volume. Verifique se os códigos {'codigo': 8186, 'territorial_level': '1', 'ibge_territorial_code': 'all', 'variable': '11709'} estão ativos em https://sidra.ibge.gov.br/home/pms/brasil.


,ipca,custo_m2,pesquisa_industrial_mensal
2000-01-01,0.62,NaN,NaN
2000-02-01,0.13,NaN,NaN
2000-03-01,0.22,NaN,NaN
2000-04-01,0.42,NaN,NaN
2000-05-01,0.01,NaN,NaN


## Method datas_ibge_link

In [19]:
DATA_INICIO = "2010-01-01"

indicadores_ibge_link = {
    "capital_fixo": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93406/d/v6561%201/l/v,p%2Bc11255,t",
    "producao_industrial_manufatureira": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8158.xlsx&terr=N&rank=-&query=t/8158/n1/all/v/11599/p/all/c543/129278/d/v11599%205/l/v,p%2Bc543,t",
    "soja": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39443/l/v,p%2Bc48,t",
    "milho_1": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39441/l/v,p%2Bc48,t",
    "milho_2": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39442/l/v,p%2Bc48,t",
    "pib": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6564/p/all/c11255/90707/d/v6564%201/l/v,p,t%2Bc11255&verUFs=false&verComplementos2=false&verComplementos1=false&omitirIndentacao=false&abreviarRotulos=false&exibirNotas=false&agruparNoCabecalho=false",
}

economic_brazil = EconomicDataAsync(codes_ibge_link=indicadores_ibge_link, 
                     start_date=DATA_INICIO)

dados = economic_brazil.datas_ibge_link_async()

In [20]:
dados.head()

,capital_fixo,producao_industrial_manufatureira,soja,milho_1,milho_2,pib
2010-01-01,12.9,89.55269,66137344.0,33424815.0,17996575.0,2.5
2010-02-01,12.9,91.15047,66941524.0,33735243.0,18620733.0,2.5
2010-03-01,29.0,114.92197,67350136.0,33486684.0,19095737.0,2.2
2010-04-01,29.0,104.58762,67913643.0,33830625.0,19457468.0,2.2
2010-05-01,29.0,110.27518,68131230.0,33577605.0,19569483.0,2.2


# Method datas_ipeadata

In [21]:
%%timeit
DATA_INICIO = "2010-01-01"

codigos_ipeadata= {
    "taja_juros_ltn": "ANBIMA12_TJTLN1212",
    "imposto_renda": "SRF12_IR12",
    "ibovespa": "ANBIMA12_IBVSP12",
    "consumo_energia": "ELETRO12_CEET12",
    "brent_fob": "EIA366_PBRENT366",
}


economic_brazil = EconomicDataAsync(codes_ipeadata=codigos_ipeadata, 
                                start_date=DATA_INICIO)

dados = economic_brazil.datas_ipeadata_async()

dados.head()

2.1 s ± 351 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Method datas_fred

In [22]:
DATA_INICIO = "2010-01-01"
codigos_fred = {
    "nasdaq100": "NASDAQ100",
    "taxa_cambio_efetiva": "RBBRBIS",
    "cboe_nasdaq": "VXNCLS",
    "taxa_juros_interbancaria": "IRSTCI01BRM156N",
    "atividade_economica_eua": "USPHCI",}

economic_brazil = EconomicDataAsync(codes_fred=codigos_fred, start_date=DATA_INICIO)

dados = economic_brazil.datas_fred_async()

dados.head()

,nasdaq100,taxa_cambio_efetiva,cboe_nasdaq,taxa_juros_interbancaria,atividade_economica_eua
2010-01-01,NaN,160.61,NaN,8.75,97.29
2010-02-01,1760.72,157.74,24.33,8.75,97.38
2010-03-01,1846.40,162.82,19.59,8.75,97.50
2010-04-01,1959.56,166.02,18.76,8.80,97.73
2010-05-01,NaN,165.54,NaN,9.50,98.20
